- 패키지 설치

In [ ]:
!pip install mxnet
!pip install gluonnlp
!pip install sentencepiece
!pip install transformers # 4.39.3
!pip install torch
!pip install kobert-transformers
#깃허브에서 KoBERT 파일 로드
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

- 라이브러리 설치

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
# error시 /usr/local/lib/python3.10/dist-packages/mxnet/numpy/utils.py in <module>의
# bool = onp.bool 을 bool = bool 로 변경하고 save (ALT+S)하고 다시 수행
import numpy as np
from tqdm import tqdm, tqdm_notebook

# ★ Hugging Face를 통한 모델 및 토크나이저 Import
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

- Fine Tuning 할 감성사전을 가져와서 입력 데이터셋에 맞게 조정

In [ ]:
import pandas as pd

# 감성사전을 딕셔너리로 불러와서 문장을 [[문장, 라벨],] 형태로 만들기
file_path = '/content/finance_data.csv'  # 파일의 경로를 적절히 수정하세요.
pd = pd.read_csv(file_path)
label_list = []
for index, row in pd.iterrows():
    if row[0] == 'negative':
      row[0] = 0
    elif row[0] == 'neutral':
      row[0] = 1
    elif row[0] == 'positive':
      row[0] = 2
    label_list.append([row[2], row[0]])
    # if index == 50:
    #   break

#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(label_list, test_size=0.25, random_state=0)
print(len(dataset_train))
print(len(dataset_test))

- 가져온 감정사전의 문장을 토큰화, 패딩 

In [ ]:
# 문장 토큰화, 패딩 클래스 (BERT 기반 모델에 대한 데이터 준비 과정을 캡슐화하여 PyTorch 파이프라인에 데이터셋을 쉽게 통합할 수 있도록 )
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

- 감정사전의 문장을 Bert학습용 입력 데이터셋으로 토큰화, 패딩 

In [ ]:
# 문장 토큰화, 패딩 클래스

# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

# 이제 버트토크나이저와 위에서 정의한 클래스를 적용해 토큰화와 패딩을 해준다.
tok = tokenizer.tokenize

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

# torch 형식의 dataset을 만들어 입력 데이터셋의 전처리 마무리
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size = batch_size, num_workers = 2)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size = batch_size, num_workers = 2)

- Kobert 모델 구현

In [17]:
# Kobert 모델 구현
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 3,   # 감정 클래스 수로 조정
                 dr_rate = None,
                 params = None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p = dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict = False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

- optimizer와 schedule 설정

In [ ]:
# optimizer 최신 사용 설정
from torch.optim import AdamW

# optimizer와 schedule 설정

# BERT  모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate = 0.5).to(device)

optimizer = AdamW(model.parameters(), lr=learning_rate)

# optimizer와 schedule 설정 (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]


optimizer = AdamW(optimizer_grouped_parameters, lr = learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 loss function

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps = warmup_step, num_training_steps = t_total)

# calc_accuracy : 정확도 측정을 위한 함수
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

train_dataloader

- 모델 학습

In [ ]:
# 학습
import os

# num_epochs = 1
start = 0
# 이전에 저장된 모델이 있는지 확인하고 있으면 로드
if os.path.exists(f'model_epoch_{start}.pt'):
    model.load_state_dict(torch.load(f'model_epoch_{start}.pt'))


# 총 num_epochs 회의 epoch를 반복합니다.
for e in range(start, num_epochs):
    train_acc = 0.0
    test_acc = 0.0

    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))


    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    # 모델 저장
    torch.save(model.state_dict(), f'model_epoch_{e+1}.pt')
    print(f'{e+1}회 모델저장 완료')
    # files.download('model_epoch_{start+1}.pt')

    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

- 학습된 모델을 저장하여 PC에 다운로드 ( 코랩 세션이 끊어지면 학습내용 초기화됨, 저장한 모델도 삭제됨 )

In [ ]:
# 파라미터, 옵티마이저(Optimizer), 에포크, 스코어 등 모든 상태를 저장
torch.save(model, 'model_5.pt')

# 가중치만 저장
torch.save(model.state_dict(), f'model_epoch_{e+1}.pt')

# local PC로 다운로드
torch.save(model, 'model_corab.pt')
files.download('/content/model_epoch_5.pt')

- 학습된 모델의 평가

In [3]:
# 훈련 결과 검증 함수

def predict(predict_sentence): # input = 감정분류하고자 하는 sentence

    model.load_state_dict(torch.load('model_epoch_5.pt'))

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False) # 토큰화한 문장
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size = batch_size, num_workers = 5) # torch 형식 변환

    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length = valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval = []
        for i in out: # out = model(token_ids, valid_length, segment_ids)
            logits = i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("완전 쉣 감정이")
            elif np.argmax(logits) == 1:
                test_eval.append("당췌 엥? 감정이")
            elif np.argmax(logits) == 2:
                test_eval.append("개 좋아! 감정이")

        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

- 실제 데이터 입력하여 강정분류 검증

In [ ]:
# 저장한 모델 불러와서 문장 테스트

device = torch.device('cpu')
model = BERTClassifier(bertmodel,  dr_rate = 0.5).to(device)
model = torch.load(r'/content/model_5.pt', device)
model.eval()

end = 1
while end == 1 :
    sentence = input("입력 : ")
    if sentence == "0" :
        break
    predict(sentence)
    print("\n")

In [ ]:
# torch.nn.Module.parameters() : 모델이 가지고 있는 가중치와 편향을 순서대로 보여준다.
for para in model.parameters() :
    print(para)

# torch.nn.Module.named_parameters() : 모델이 가지고 있는 (layer의 '이름', 해당 layer의 'parameter')를 순서대로 보여준다.
for name, para in model.named_parameters() :
    print(name)
    print(para)

In [ ]:
# model.children() : 계층의 특성을 요약해서 보여준다.
for child in model.children() :
    print(child)

# model.named_children() : 각 계층의 이름과 요약 정보를 보여준다
for name, child in model.named_children() :
    print(name, child)